In [4]:
import numpy as np

un_o_matrix = np.mat([[1, 2, 3, 4], [2, 5, 7, 2], [2, 4, 3, 5]])
row_len, column_len = un_o_matrix.shape
print(row_len)

3


In [31]:
delta_x = np.mat([[1, 2, 2, 2], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
print(delta_x[0, :])
print(np.linalg.norm(delta_x[0, :]))
x1 = np.log(np.linalg.norm(delta_x[0, :]))
print(x1)

[[1 2 2 2]]
3.605551275463989
1.2824746787307684
